In [156]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException

import time
#brew install geckodrivee

In [393]:
def error_parse(error):
    if "trustee" in error:
        return "CUTrustee"
    return error

In [ ]:
driver = webdriver.Firefox()

In [423]:
driver.get("https://tools.alumni.columbia.edu/uni/SelectUser.aspx")


###to shut things down:
# elem.clear()
# elem.send_keys("pycon")
# elem.send_keys(Keys.RETURN)
# assert "No results found." not in driver.page_source
# driver.close()

###getting other things done:
# dir(By)
# print(driver.title)
# print(dir(driver))
# 
# ids = elem.find_elements_by_xpath('//*[@id]')


In [424]:
elem = driver.find_element_by_id("container")

In [425]:
firstname = elem.find_elements_by_id("ctl00_rightcol_ph_first_tb")


In [426]:
lastname = elem.find_elements_by_id("ctl00_rightcol_ph_last_tb")
searchbutton=elem.find_elements_by_id("ctl00_rightcol_ph_Button2")
school=elem.find_elements_by_id("ctl00_rightcol_ph_school_ddl_Input")

In [427]:
alumni = []
for class_year in range(1970,1989):
    for firstchar in "abcdefghijklmnopqrstuvwxyz":
        for lastchar in "abcdefghijklmnopqrstuvwxyz":
            school=elem.find_elements_by_id("ctl00_rightcol_ph_school_ddl_Input")
            school[0].click()
            college=driver.find_element_by_id("ctl00_rightcol_ph_school_ddl_c6").click()
            year = elem.find_element_by_id("ctl00_rightcol_ph_year_ddl")
            year.click()
            year.send_keys(str(class_year))

            firstname[0].send_keys(firstchar)
            lastname[0].send_keys(lastchar)
            searchbutton[0].send_keys(Keys.ENTER)
            time.sleep(5)
            try:
                table = driver.find_element_by_id("ctl00_rightcol_ph_result_grid")
            except NoSuchElementException:
                time.sleep(5)
                table = driver.find_element_by_id("ctl00_rightcol_ph_result_grid")
            people = table.find_elements(By.TAG_NAME, "tr")

            for person in people[1:]:

                info = person.text.split(" ")
                alumni.append([" ".join(info[:3]),info[-2],info[-1]])

            for record in alumni:
                link = driver.find_element_by_link_text(record[0])
                link.click()
                try:
                    emails = driver.find_element_by_id("ctl00_leftcol_ph_emails_rbl_0")
                except NoSuchElementException:
                    record.append(email)

                    try:
                        error_message = driver.find_element_by_id("ctl00_rightcol_ph_ErrorPnl")
                    except NoSuchElementException:
                        error_message = "other error"
                    record.append(error_parse(email))
                    driver.back()
                    continue

                emails.click()
                email = emails.get_attribute("value")
                record.append(email)
                driver.back()
            driver.get("https://tools.alumni.columbia.edu/uni/SelectUser.aspx")
            elem = driver.find_element_by_id("container")
            firstname = elem.find_elements_by_id("ctl00_rightcol_ph_first_tb")
            lastname = elem.find_elements_by_id("ctl00_rightcol_ph_last_tb")
            searchbutton=elem.find_elements_by_id("ctl00_rightcol_ph_Button2")
            school=elem.find_elements_by_id("ctl00_rightcol_ph_school_ddl_Input")

NoSuchElementException: Message: Unable to locate element: Albert H. Bergeret


In [430]:
alumni

[['Albert H. Bergeret', "'70CC", 'ahb30', 'al••••@nygasp.org'],
 ['Arthur D. Bramble',
  "'70CC",
  'adb56',
  'al••••@nygasp.org',
  'al••••@nygasp.org'],
 ['Andre E. Broussard', "'70CC", 'aeb58', 'ae•••@caa.columbia.edu'],
 ['Allen E. Buchanan',
  "'70CC",
  'aeb61',
  'ae•••@caa.columbia.edu',
  'ae•••@caa.columbia.edu'],
 ['Alan I. Charney', "'70CC", 'aic16'],
 ['Anthony P. Ciambrone', "'70CC", 'apc25'],
 ['Anthony J. Cicconi', "'70CC", 'ajc62']]

In [377]:
"in" in "helloin"

True

In [336]:
alumni

[['Laura Rose Cantrell', "'89CC", 'lrc19'],
 ['Lisa Landau Carnoy', "'89CC", 'll381'],
 ['Louis H. Claudio', "'89CC", 'lhc2107'],
 ['Leslie Ann Coe', "'89CC", 'lck19']]

In [289]:
laura = people[1]
link = laura.find_element_by_link_text("Laura Rose Cantrell")
link.click()

In [291]:
people[2].text

StaleElementReferenceException: Message: The element with the reference cbe6dcb5-9aae-4c14-81f9-d985c624645b is stale; either its node document is not the active document, or it is no longer connected to the DOM


In [223]:
emails = driver.find_element_by_id("ctl00_leftcol_ph_emails_rbl_0")
emails.click()
emails.get_attribute("value")

In [272]:
people[1]

<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="b9c1ab03-fc26-4eac-b5e6-6a82f20cc8b7", element="37dcabb0-a2ae-449f-b2b7-c43dd1132f2e")>

In [273]:
people[1].text

StaleElementReferenceException: Message: The element with the reference 37dcabb0-a2ae-449f-b2b7-c43dd1132f2e is stale; either its node document is not the active document, or it is no longer connected to the DOM


In [229]:
laura.id

'37dcabb0-a2ae-449f-b2b7-c43dd1132f2e'

In [243]:
link = laura.find_element_by_link_text("Laura Rose Cantrell")
link.click()

In [246]:
emails = driver.find_element_by_id("ctl00_leftcol_ph_emails_rbl_0")




In [251]:
emails.__class__

selenium.webdriver.firefox.webelement.FirefoxWebElement

In [255]:
emails.click()

In [267]:
emails.get_attribute("value")

'la•••@wfmu.org'

In [235]:
dir(laura)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_execute',
 '_id',
 '_parent',
 '_upload',
 '_w3c',
 'anonymous_children',
 'clear',
 'click',
 'find_anonymous_element_by_attribute',
 'find_element',
 'find_element_by_class_name',
 'find_element_by_css_selector',
 'find_element_by_id',
 'find_element_by_link_text',
 'find_element_by_name',
 'find_element_by_partial_link_text',
 'find_element_by_tag_name',
 'find_element_by_xpath',
 'find_elements',
 'find_elements_by_class_name',
 'find_elements_by_css_selector',
 'find_elements_by_id',
 'find_elements_by_link_text',
 'find_elements_by_name',
 'find_elements_by_partial_link_text',
 'find_elements_by_tag_name',
 'find_

In [61]:
##getting the components on the page:
ids = elem.find_elements_by_xpath('//*[@id]')
for ii in ids:
    #print ii.tag_name
    print(ii.get_attribute('id'))

aspnetForm
__EVENTTARGET
__EVENTARGUMENT
__VIEWSTATE
__VIEWSTATEGENERATOR
__VIEWSTATEENCRYPTED
__EVENTVALIDATION
container
page
page-inner
header
branding
brand-wrapper
brand-wrapper-inner
branded-site-name
site-name
block-cu-admin1-mainnavigation
block-cu-admin1-mainnavigation-menu
site-nav
site-nav-second
mobile-nav
mobile-nav-container
sidebar-left
sidebar-left-inner
content-area
sidebar-left
sidebar-left-inner
ctl00_rightcol_ph_RadTabStrip1
ctl00_rightcol_ph_RadTabStrip1StyleSheetHolder
ctl00_rightcol_ph_RadTabStrip1_Tab1
ctl00_rightcol_ph_RadTabStrip1_Tab2
ctl00_rightcol_ph_RadTabStrip1_Hidden
ctl00_rightcol_ph_RadMultiPage1
ctl00_rightcol_ph_name_view
ctl00_rightcol_ph_byname
ctl00_rightcol_ph_first_tb
ctl00_rightcol_ph_last_tb
ctl00_rightcol_ph_school_ddl_wrapper
ctl00_rightcol_ph_school_ddl
ctl00_rightcol_ph_school_ddl_Input
ctl00_rightcol_ph_school_ddl_Image
ctl00_rightcol_ph_school_ddl_DropDownPlaceholder
ctl00_rightcol_ph_school_ddl_DropDown
ctl00_rightcol_ph_school_ddl_c0
c